# Fooocus: код с GitHub + модели с Яндекс.Диска

**Схема:** репозиторий на GitHub (код), модели лежат на **Яндекс.Диске**. В Colab клонируем код и подключаем папку с моделями через rclone.

Подходит, если на Google Drive мало места (например, 15 ГБ), а на Яндексе — больше.

## Перед запуском

1. Выгрузите **код** Fooocus на GitHub (без папок `models`, `venv`).
2. Один раз залейте папку **моделей** (`models 12.07.29` или `models`) на **Яндекс.Диск**.
3. На **вашем компьютере** (с браузером) выполните один раз и сохраните токен:
   ```bash
   rclone authorize "yandex"
   ```
   Откроется браузер, войдите в Яндекс — в консоли появится длинная строка (token). Скопируйте её целиком.
4. **Runtime → Change runtime type → GPU** (T4).
5. Ниже укажите: репозиторий GitHub, путь на Яндекс.Диске к папке с моделями и вставьте токен.
6. Запустите ячейки по порядку.

In [ ]:
# Ваш репозиторий Fooocus на GitHub
GITHUB_REPO = "https://github.com/lolikoni/Focus.git"

# Путь на Яндекс.Диске к папке, где лежат модели.
# Это путь ВНУТРИ диска, например: "Fooocus/models 12.07.29" или "AI-modeli/Fooocus/models 12.07.29"
# Без ведущего слеша. Корень диска — просто имя папки, например "Fooocus_models".
# Путь как в веб-интерфейсе Яндекс.Диска (хлебные крошки: Мак → Мои проекты → AI-modeli → Fooocus → models 12.07.29)
YANDEX_MODELS_PATH = "Мак/Мои проекты/AI-modeli/Fooocus/models 12.07.29"

# Токен от rclone authorize "yandex" (вставьте строку целиком)
# Получить: на своём компе в терминале: rclone authorize "yandex"
# Удобно вставить в тройных кавычках:
YANDEX_TOKEN = """{"access_token":"...","token_type":"OAuth","expiry":"..."}"""

In [ ]:
# Клонируем код с GitHub
import os
import shutil

REPO_DIR = "/content/Fooocus"
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

get_ipython().system(f'git clone --depth 1 {GITHUB_REPO} {REPO_DIR}')

FOOCUS_ROOT = REPO_DIR
print(f"Код склонирован: {FOOCUS_ROOT}")

In [ ]:
# Устанавливаем rclone и настраиваем доступ к Яндекс.Диску
!curl -O https://downloads.rclone.org/rclone-current-linux-amd64.zip
!unzip -o -q rclone-current-linux-amd64.zip
!cp rclone-*-linux-amd64/rclone /usr/bin/
!chmod 755 /usr/bin/rclone

# Создаём конфиг rclone с вашим токеном (запись в файл — без проблем с кавычками)
import os
os.makedirs("/root/.config/rclone", exist_ok=True)
config_content = "[yandex]\ntype = yandex\ntoken = " + YANDEX_TOKEN + "\n"
with open("/root/.config/rclone/rclone.conf", "w") as f:
    f.write(config_content)
print("rclone настроен для Яндекс.Диска.")

In [ ]:
# Монтируем папку с моделями с Яндекс.Диска (нужен FUSE; в Colab обычно есть)
YANDEX_MOUNT = "/content/yandex_models"
os.makedirs(YANDEX_MOUNT, exist_ok=True)

# Запускаем mount в фоне (rclone mount держит процесс)
!fusermount -uz $YANDEX_MOUNT 2>/dev/null || true
get_ipython().system(f'rclone mount yandex:{YANDEX_MODELS_PATH} {YANDEX_MOUNT} --vfs-cache-mode full --daemon')

import time
time.sleep(2)
if not os.path.isdir(YANDEX_MOUNT) or not os.listdir(YANDEX_MOUNT):
    print("Ожидание монтирования...")
    time.sleep(3)

if os.path.isdir(YANDEX_MOUNT) and os.listdir(YANDEX_MOUNT):
    print(f"Яндекс.Диск смонтирован: {YANDEX_MOUNT}")
    # Ищем checkpoints внутри смонтированной папки
    path_checkpoints = os.path.join(YANDEX_MOUNT, "checkpoints")
    if os.path.isdir(path_checkpoints):
        models_base = "models 12.07.29"
    else:
        models_base = "models"
        path_checkpoints = os.path.join(YANDEX_MOUNT, "checkpoints")
    if os.path.isdir(path_checkpoints):
        models = [f for f in os.listdir(path_checkpoints) if f.endswith((".safetensors", ".ckpt"))]
        print(f"Чекпоинтов: {len(models)}")
        for m in models[:8]:
            print(f"  - {m}")
else:
    raise RuntimeError(
        f"Не удалось смонтировать Яндекс.Диск. Проверьте YANDEX_MODELS_PATH и токен. "
        "Путь должен быть без ведущего /, например: Fooocus/models 12.07.29"
    )

In [ ]:
# Подключаем смонтированную папку к Fooocus (симлинк)
models_folder_name = "models 12.07.29" if "12.07.29" in YANDEX_MODELS_PATH else "models"
target_models = os.path.join(FOOCUS_ROOT, models_folder_name)
if os.path.exists(target_models):
    os.remove(target_models) if os.path.islink(target_models) else shutil.rmtree(target_models)
os.symlink(YANDEX_MOUNT, target_models)
print(f"Модели подключены: {target_models} -> {YANDEX_MOUNT}")

In [ ]:
# Прописываем пути в config.txt для Colab
import json

models_dir = os.path.join(FOOCUS_ROOT, models_folder_name)
config_path = os.path.join(FOOCUS_ROOT, "config.txt")

paths = {
    "path_checkpoints": [os.path.join(models_dir, "checkpoints")],
    "path_loras": [os.path.join(models_dir, "loras")],
    "path_embeddings": os.path.join(models_dir, "embeddings"),
    "path_vae_approx": os.path.join(models_dir, "vae_approx"),
    "path_vae": os.path.join(models_dir, "vae"),
    "path_outputs": os.path.join(FOOCUS_ROOT, "outputs"),
}
for key, val in paths.items():
    if isinstance(val, list):
        for v in val:
            os.makedirs(v, exist_ok=True)
    else:
        os.makedirs(val, exist_ok=True)

config = {} if not os.path.isfile(config_path) else json.load(open(config_path, "r", encoding="utf-8"))
config.update(paths)
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4, ensure_ascii=False)
print("config.txt обновлён.")

In [ ]:
# Запуск Fooocus
os.chdir(FOOCUS_ROOT)

!pip install -q pygit2==1.15.1
!python launch.py --share --always-high-vram